In [38]:
from xformers.ops.fmha.attn_bias import (
    AttentionBias,
    BlockDiagonalCausalMask,
    BlockDiagonalCausalWithOffsetPaddedKeysMask,
    BlockDiagonalMask,
)

import pandas as pd

In [39]:
## BlockDiagonalCausalMask

seqlens = [7, 5, 6]
sliding_window_size = 3

mask = BlockDiagonalCausalMask.from_seqlens(seqlens).make_local_attention(sliding_window_size)

batch_size = 1
total_seq_len = sum(seqlens)
mask_tensor = mask.materialize((batch_size, total_seq_len, total_seq_len))

pd.DataFrame(mask_tensor[0, :, :].numpy())

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,0.0,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf
1,0.0,0.0,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf
2,0.0,0.0,0.0,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf
3,-inf,0.0,0.0,0.0,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf
4,-inf,-inf,0.0,0.0,0.0,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf
5,-inf,-inf,-inf,0.0,0.0,0.0,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf
6,-inf,-inf,-inf,-inf,0.0,0.0,0.0,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf
7,-inf,-inf,-inf,-inf,-inf,-inf,-inf,0.0,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf
8,-inf,-inf,-inf,-inf,-inf,-inf,-inf,0.0,0.0,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf
9,-inf,-inf,-inf,-inf,-inf,-inf,-inf,0.0,0.0,0.0,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf


In [40]:
## BlockDiagonalMask

q_seqlens = [3, 5]
kv_seqlens = [10, 8] # (3 + 7, 5 + 3)
sliding_window_size = 3

mask = BlockDiagonalMask.from_seqlens(q_seqlens, kv_seqlens).make_local_attention_from_bottomright(sliding_window_size)

batch_size = 1
total_seq_len = sum(q_seqlens)
total_kv_seq_len = sum(kv_seqlens)
mask_tensor = mask.materialize((batch_size, total_seq_len, total_kv_seq_len))

pd.DataFrame(mask_tensor[0, :, :].numpy())

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,-inf,-inf,-inf,-inf,-inf,0.0,0.0,0.0,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf
1,-inf,-inf,-inf,-inf,-inf,-inf,0.0,0.0,0.0,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf
2,-inf,-inf,-inf,-inf,-inf,-inf,-inf,0.0,0.0,0.0,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf
3,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,0.0,0.0,0.0,-inf,-inf,-inf,-inf
4,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,0.0,0.0,0.0,-inf,-inf,-inf
5,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,0.0,0.0,0.0,-inf,-inf
6,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,0.0,0.0,0.0,-inf
7,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,0.0,0.0,0.0


In [41]:
## BlockDiagonalCausalWithOffsetPaddedKeysMask

q_seqlen = [1, 1]
kv_seq_len = [3, 5]
kv_padding = 6

mask = BlockDiagonalCausalWithOffsetPaddedKeysMask.from_seqlens(q_seqlen=q_seqlen, kv_padding=kv_padding, kv_seqlen=kv_seq_len)

batch_size = 1
total_seq_len = sum(q_seqlen)
total_kv_seq_len = kv_padding * len(kv_seq_len)

mask_tensor = mask.materialize((batch_size, total_seq_len, total_kv_seq_len))

pd.DataFrame(mask_tensor[0, :, :].numpy())

,0,1,2,3,4,5,6,7,8,9,10,11
0,0.0,0.0,0.0,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf
1,-inf,-inf,-inf,-inf,-inf,-inf,0.0,0.0,0.0,0.0,0.0,-inf
